#Install lib

In [1]:
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 83.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 116.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 20.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.3 MB/s eta 0:00:00


In [3]:
#!apt install git-lfs

#Enviroment

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# This flag is the difference between SQUAD v1 or 2 (if you're using another dataset, it indicates if impossible answers are allowed or not).
squad_v2 = True
#model_checkpoint = "xlm-roberta-base"
model_checkpoint = "deepset/tinyroberta-squad2"
batch_size = 16

#Preprocessing the training data

In [9]:
import json
from pathlib import Path
import torch

In [10]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [12]:
def read_squad_train(path,data_index,context_size):
  """
  Read data format squad 
  -> return contexts, questions, answers of sentenses
  """
  path = Path(path)
  with open(path, 'rb') as f:
      squad_dict = json.load(f)

  contexts = []
  questions = []
  answers = []
  size=0
  for group in squad_dict['data'][data_index-1:data_index]:
      for passage in group['paragraphs']:
        if size <= context_size:
          context = passage['context']
          for qa in passage['qas']:
              question = qa['question']
              for answer in qa['answers']:
                if answer['answer_start']>=0:
                  contexts.append(context)
                  questions.append(question)
                  answers.append(answer)
          size+=1
        else: break

  return contexts, questions, answers
def read_squad_val(path):
  """
  Read data format squad 
  -> return contexts, questions, answers of sentenses
  """
  path = Path(path)
  with open(path, 'rb') as f:
      squad_dict = json.load(f)

  contexts = []
  questions = []
  answers = []
  for group in squad_dict['data']:
      for passage in group['paragraphs']:
          context = passage['context']
          for qa in passage['qas']:
              question = qa['question']
              for answer in qa['answers']:
                  contexts.append(context)
                  questions.append(question)
                  answers.append(answer)

  return contexts, questions, answers
def add_token_positions(encodings, answers):
  """
  Convert answer start- end from string to token
  """
  start_positions = []
  end_positions = []
  for i in range(len(answers)):
    start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
    end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))
    # if None, the answer passage has been truncated
    if start_positions[-1] is None:
        start_positions[-1] = tokenizer.model_max_length
    if end_positions[-1] is None:
        end_positions[-1] = tokenizer.model_max_length
  encodings.update({'start_positions': start_positions, 'end_positions': end_positions})
def add_end_idx(answers, contexts):
  """
  Squad format chỉ có id start của câu trả lời
  -> Hàm này dùng để tạo id end của câu trả lời
  """
  for answer, context in zip(answers, contexts):
    gold_text = answer['text']
    start_idx = answer['answer_start']
    end_idx = start_idx + len(gold_text)

    # sometimes squad answers are off by a character or two – fix this
    if context[start_idx:end_idx] == gold_text:
        answer['answer_end'] = end_idx
    elif context[start_idx-1:end_idx-1] == gold_text:
        answer['answer_start'] = start_idx - 1
        answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
    elif context[start_idx-2:end_idx-2] == gold_text:
        answer['answer_start'] = start_idx - 2
        answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

In [ ]:
train_contexts, train_questions, train_answers = read_squad_train('/content/drive/MyDrive/CS221/DoAnCK/Dataset/squad_vi.json', 1,3500)
#val_contexts, val_questions, val_answers = read_squad_train('/content/drive/MyDrive/CS221/DoAnCK/Dataset/squad_vi.json', 3000, 3500)
add_end_idx(train_answers, train_contexts)
#add_end_idx(val_answers, val_contexts) 

#train
new_train_contexts=[]
new_train_questions=[]
new_train_answers=[]
for i in range(len(train_answers)):
  if train_answers[i]['answer_start']!=train_answers[i]['answer_end']:
    new_train_contexts.append(train_contexts[i])
    new_train_questions.append(train_questions[i])
    new_train_answers.append(train_answers[i])
(train_contexts, train_questions, train_answers)=(new_train_contexts, new_train_questions, new_train_answers)
(new_train_contexts, new_train_questions, new_train_answers)=(0,0,0)

#chia train val
val_contexts, val_questions, val_answers = train_contexts[12000:], train_questions[12000:], train_answers[12000:]
train_contexts, train_questions, train_answers = train_contexts[:12000], train_questions[:12000], train_answers[:12000]


train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)


Tải data lên pytorch

In [16]:
class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

In [12]:
train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

#Load pretrain

In [13]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

In [14]:
model

RobertaForQuestionAnswering(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm):

#Train

In [15]:
from torch.utils.data import DataLoader
from transformers import AdamW

In [16]:
epochs = 10
batch_size=8
train_count = len(train_dataset)
val_count = len(val_dataset)
PATH = "/content/drive/MyDrive/CS221/DoAnCK/Model/best_model.model"

###Train with trainer

###10 epoch

In [17]:
import numpy as np

In [18]:
args = TrainingArguments(
    f"/content/drive/MyDrive/CS221/DoAnCK/Logs/MiniBert-ViSquad",#Tên folder chứa log
    evaluation_strategy = "epoch",#Đánh giá sau mỗi epoch
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01
    #push_to_hub=True,
)

In [22]:
def f1_score(start_pred,end_pred,start_true,end_true):
  a=np.arange(int(start_pred),int(end_pred+1), step=1)
  b=np.arange(int(start_true),int(end_true+1), step=1)
  true_count = len(np.intersect1d(a,b))
  if ((int(end_true)-int(start_true)) == 0):
    recall = 0
  else:
    recall = true_count/(int(end_true)-int(start_true))
  if ((int(end_pred)-int(start_pred)) == 0):
    precision = 0
  else:
    precision= true_count/(int(end_pred)-int(start_pred))
  if (precision + recall == 0):
    return 0
  f1=2*(precision*recall)/(precision+recall)
  return f1

In [23]:
def compute_metrics(eval_pred):    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    count = 0
    for i in range(len(predictions[0])):
      if (predictions[0][i] == labels[0][i] and predictions[1][i] == labels[1][i]):
        count += 1
    f1 = 0
    for i in range(len(predictions[0])):
      f1 += f1_score(predictions[0][i], predictions[1][i], labels[0][i], labels[1][i])

    return {'exact_match': round(count/len(predictions[0]), 3), 'f1_score': round(f1/len(predictions[0]),3)} #metric.compute(predictions=predictions, references=labels)

In [21]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    #data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [22]:
trainer.train()
trainer.save_model("/content/drive/MyDrive/CS221/DoAnCK/Model/Minibert-viSquad")

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 12000
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 15000
  Number of trainable parameters = 81529346
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Exact Match,F1 Score
1,3.125700,3.119982,0.169000,0.257000
2,2.710200,2.813204,0.207000,0.328000
3,2.346900,2.815731,0.240000,0.348000
4,2.092500,2.915711,0.253000,0.364000
5,1.808900,2.900170,0.251000,0.369000
6,1.641000,3.015230,0.259000,0.380000
7,1.423300,3.233148,0.259000,0.397000
8,1.244800,3.526010,0.254000,0.379000
9,1.116500,3.481797,0.262000,0.389000
10,1.019200,3.775353,0.259000,0.381000


***** Running Evaluation *****
  Num examples = 1947
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/CS221/DoAnCK/Logs/MiniBert-ViSquad/checkpoint-1500
Configuration saved in /content/drive/MyDrive/CS221/DoAnCK/Logs/MiniBert-ViSquad/checkpoint-1500/config.json
Model weights saved in /content/drive/MyDrive/CS221/DoAnCK/Logs/MiniBert-ViSquad/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/CS221/DoAnCK/Logs/MiniBert-ViSquad/checkpoint-1500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/CS221/DoAnCK/Logs/MiniBert-ViSquad/checkpoint-1500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1947
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/CS221/DoAnCK/Logs/MiniBert-ViSquad/checkpoint-3000
Configuration saved in /content/drive/MyDrive/CS221/DoAnCK/Logs/MiniBert-ViSquad/checkpoint-3000/config.json
Model weights saved in /content/drive/MyDrive/CS221/DoAnCK/Logs/MiniB

TrainOutput(global_step=15000, training_loss=1.8731856567382812, metrics={'train_runtime': 6679.9282, 'train_samples_per_second': 17.964, 'train_steps_per_second': 2.246, 'total_flos': 1.567837200384e+16, 'train_loss': 1.8731856567382812, 'epoch': 10.0})

Evaluation

#Test

In [142]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import AutoTokenizer

model = AutoModelForQuestionAnswering.from_pretrained("/content/drive/MyDrive/CS221/DoAnCK/model/MiniBert-ViSquad/checkpoint-15000") 
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/CS221/DoAnCK/Logs/MiniBert-ViSquad/checkpoint-15000")

OSError: ignored

In [89]:
test_contexts, test_questions, test_answers = read_squad_val('/content/drive/MyDrive/CS221/DoAnCK/Dataset/Mailong25/train-v2.0.json')
add_end_idx(test_answers, test_contexts)
new_test_contexts=[]
new_test_questions=[]
new_test_answers=[]
for i in range(len(test_answers)):
  if test_answers[i]['answer_start']!=test_answers[i]['answer_end']:
    new_test_contexts.append(test_contexts[i])
    new_test_questions.append(test_questions[i])
    new_test_answers.append(test_answers[i])
(test_contexts, test_questions, test_answers)=(new_test_contexts, new_test_questions, new_test_answers)
test_encodings = tokenizer(test_contexts, test_questions, truncation=True, padding=True, return_offsets_mapping=True,)
add_token_positions(test_encodings, test_answers)
test_dataset = SquadDataset(test_encodings)

In [87]:
test_encodings[0]

Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [63]:
test_contexts[0]

'Narendra Damodardas Modi ( tiếng Gujarat : નરેન્દ્ર દામોદરદાસ મોદી , [ nəreːnd̪rə d̪ɑːmoːd̪ərəd̪ɑːsə moːd̪iː ] ( nghe ) , tiếng Hindi : नरेन्द्र दामोदरदास मोदी , sinh ngày 17 tháng 9 năm 1950 ) là Thủ tướng Ấn Độ thứ 15 . Ông là thủ lĩnh của đảng Bharatiya Janata ( đảng Nhân dân ) , từng giữ chức Thủ tịch bộ trưởng Gujarat từ năm 2001 đến năm 2014 . Ông đại diện cho Varanasi tại Lok Sabha .'

In [65]:
test_contexts[0][0:24]

'Narendra Damodardas Modi'

In [64]:
test_answers[0]

{'answer_start': 0, 'text': 'Narendra Damodardas Modi', 'answer_end': 24}

In [19]:
args = TrainingArguments(
    f"/content/drive/MyDrive/CS221/DoAnCK/Logs/MiniBert-ViSquad-test",#Tên folder chứa log
    evaluation_strategy = "epoch",#Đánh giá sau mỗi epoch
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01
    #push_to_hub=True,
)

In [25]:
import numpy as np

In [26]:
trainer = Trainer(
    model,
    args,
    #train_dataset=train_dataset,
    eval_dataset=test_dataset,
    #data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 985
  Batch size = 16


{'eval_loss': 3.7535667419433594,
 'eval_exact_match': 0.241,
 'eval_f1_score': 0.527,
 'eval_runtime': 16.3206,
 'eval_samples_per_second': 60.353,
 'eval_steps_per_second': 3.799}

#Predict

In [ ]:
trainer.predict()

In [136]:
def Predict(contexts, questions, trainer):
  test_encodings = trainer.tokenizer(contexts, questions, truncation=True, padding=True, return_overflowing_tokens=True, return_offsets_mapping=True,)
  test_dataset = SquadDataset(test_encodings)
  
  result = trainer.predict(test_dataset, )
  predictions = np.argmax(result.predictions, axis =-1)
  metrics = result.metrics
  
  answers = []
  for i in range(len(contexts)):
    start = test_encodings['offset_mapping'][0][predictions[0][0]][0]
    end = test_encodings['offset_mapping'][0][predictions[1][0]][1]
    answers.append(contexts[i][start: end])
  return answers[0]

In [143]:
a = Predict(['Nam sinh ra và lớn lên ở Thủ Đức. Nam học công nghệ thông tin'], ["Nam học gì"], trainer)
a

***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `RobertaForQuestionAnswering.forward`,  you can safely ignore this message.


'Nam sinh ra và lớn lên ở Thủ Đức'

In [140]:
for i in range(0, 100, 10):
  print("Context:", test_contexts[i])
  print("Question:", test_questions[i])
  print("Trả lời:", Predict([test_contexts[i]], [test_questions[i]], trainer))
  print("\n==========")

***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `RobertaForQuestionAnswering.forward`,  you can safely ignore this message.


Context: Narendra Damodardas Modi ( tiếng Gujarat : નરેન્દ્ર દામોદરદાસ મોદી , [ nəreːnd̪rə d̪ɑːmoːd̪ərəd̪ɑːsə moːd̪iː ] ( nghe ) , tiếng Hindi : नरेन्द्र दामोदरदास मोदी , sinh ngày 17 tháng 9 năm 1950 ) là Thủ tướng Ấn Độ thứ 15 . Ông là thủ lĩnh của đảng Bharatiya Janata ( đảng Nhân dân ) , từng giữ chức Thủ tịch bộ trưởng Gujarat từ năm 2001 đến năm 2014 . Ông đại diện cho Varanasi tại Lok Sabha .
Question: Ai là thủ tướng Ấn Độ hiện tại


***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `RobertaForQuestionAnswering.forward`,  you can safely ignore this message.


Trả lời: Narendra Damodardas Modi

Context: Chia sẻ với VnExpress sau khi được Forbes công nhận là tỷ phú USD với 1,3 tỷ USD tài sản , ông Trần Đình Long - Chủ tịch HĐQT Tập đoàn Hoà Phát , thú nhận đã lâu không làm việc để vì mục tiêu kiếm một con số cụ thể bao nhiêu tiền trong ngày .
Question: Ai là chủ tịch tập đoàn Hòa Phát


***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `RobertaForQuestionAnswering.forward`,  you can safely ignore this message.


...

***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `RobertaForQuestionAnswering.forward`,  you can safely ignore this message.


***** Running Prediction *****
  Num examples = 1


Trả lời: ,

Context: “ Vài năm nay , mọi người nhìn thép rất xấu , nhưng thép vẫn được coi là bánh mì của công nghiệp và tại những nước công nghiệp hoá mới như chúng ta , nhu cầu thép vẫn còn tăng ” , đó là chia sẻ chân tình của ông Trần Đình Long , Chủ tịch HĐQT Tập đoàn Hoà Phát khi nói về những mối quan tâm của dư luận thời gian gần đây trước những biến động lớn trong ngành thép lẫn tương lai của ngành này tại Việt Nam .
Question: chủ tịch tập đoàn Hòa Phát


  Batch size = 16
The following columns in the test set don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: overflow_to_sample_mapping, offset_mapping. If overflow_to_sample_mapping, offset_mapping are not expected by `RobertaForQuestionAnswering.forward`,  you can safely ignore this message.


Trả lời: ,

